In [17]:
import requests
import pandas as pd
import hmac
import hashlib
from datetime import datetime

In [62]:
import credentials
secret_bytes = bytes(credentials.secret , 'latin-1')

In [36]:
rates = requests.get("https://api.coinbase.com/v2/exchange-rates").json()["data"]["rates"]

In [60]:
def generate_access_sign(timestamp, method, requestPath, body):
    message = timestamp + method + requestPath + body
    message_bytes = bytes(message , 'latin-1')
    signature = hmac.new(secret_bytes, message_bytes, digestmod = hashlib.sha256).hexdigest()
    return signature

In [64]:
BASE = "https://api.coinbase.com"
PATH = "/v2/accounts"
timestamp = str(round(datetime.now().timestamp()))
signature = generate_access_sign(timestamp, "GET", PATH, "")
HEADERS = {
    "CB-ACCESS-KEY": credentials.key,
    "CB-ACCESS-SIGN": signature,
    "CB-ACCESS-TIMESTAMP": timestamp
}

In [63]:
wallets = requests.get(BASE+PATH, headers=HEADERS).json()

In [65]:
requests.get("https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=1000&page=1&sparkline=false").json()

[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579',
  'current_price': 36080,
  'market_cap': 687093350325,
  'market_cap_rank': 1,
  'fully_diluted_valuation': 758114211690,
  'total_volume': 32168334376,
  'high_24h': 36646,
  'low_24h': 35487,
  'price_change_24h': -531.941315091965,
  'price_change_percentage_24h': -1.45291,
  'market_cap_change_24h': -8927593556.442505,
  'market_cap_change_percentage_24h': -1.28266,
  'circulating_supply': 19032700.0,
  'total_supply': 21000000.0,
  'max_supply': 21000000.0,
  'ath': 69045,
  'ath_change_percentage': -47.68697,
  'ath_date': '2021-11-10T14:24:11.849Z',
  'atl': 67.81,
  'atl_change_percentage': 53166.39881,
  'atl_date': '2013-07-06T00:00:00.000Z',
  'roi': None,
  'last_updated': '2022-05-06T23:58:15.023Z'},
 {'id': 'ethereum',
  'symbol': 'eth',
  'name': 'Ethereum',
  'image': 'https://assets.coingecko.com/coins/images/279/large/ether